# Installing libraries

In [ ]:
pip install google-generativeai --upgrade --quiet
# !pip install google-ai-generativelanguage==0.6.15 --force-reinstall --upgrade --quiet # when normal installation fails

# other necessary packages

!pip install -q youtube-transcript-api langchain-community langchain-openai faiss-cpu tiktoken python-dotenv langchain_google_genai --quiet


In [19]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled

from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings


from langchain_community.vectorstores import FAISS

from langchain_core.prompts import PromptTemplate

from langchain_google_genai import ChatGoogleGenerativeAI

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough,RunnableLambda

from dotenv import load_dotenv
import os

load_dotenv()


True

# Step1 Indexing
a.  (Document Ingestion)

In [20]:
video_id = "LPZh9BOjkQs"  # only the ID, not full URL
try:
    # If you don't care which language, this returns the "best" one
    transcript_list = YouTubeTranscriptApi.get_transcript(video_id, languages=["en"])

    # Flatten it to plain text
    transcript = " ".join(chunk["text"] for chunk in transcript_list) # basically here we concatenating items of genrerator by space
    print(transcript)

except Exception as e:
    print("Error:" , e)

Imagine you happen across a short movie script that describes a scene between a person and their AI assistant. The script has what the person asks the AI, but the AI's response has been torn off. Suppose you also have this powerful magical machine that can take any text and provide a sensible prediction of what word comes next. You could then finish the script by feeding in what you have to the machine, seeing what it would predict to start the AI's answer, and then repeating this over and over with a growing script completing the dialogue. When you interact with a chatbot, this is exactly what's happening. A large language model is a sophisticated mathematical function that predicts what word comes next for any piece of text. Instead of predicting one word with certainty, though, what it does is assign a probability to all possible next words. To build a chatbot, you lay out some text that describes an interaction between a user and a hypothetical AI assistant, add on whatever the use

# Step1
b. (Text Splitting)

In [21]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.create_documents([transcript])

# Step 1c & 1d - Indexing (Embedding Generation and Storing in Vector Store)

In [22]:
chunks

[Document(metadata={}, page_content="Imagine you happen across a short movie script that describes a scene between a person and their AI assistant. The script has what the person asks the AI, but the AI's response has been torn off. Suppose you also have this powerful magical machine that can take any text and provide a sensible prediction of what word comes next. You could then finish the script by feeding in what you have to the machine, seeing what it would predict to start the AI's answer, and then repeating this over and over with a growing script completing the dialogue. When you interact with a chatbot, this is exactly what's happening. A large language model is a sophisticated mathematical function that predicts what word comes next for any piece of text. Instead of predicting one word with certainty, though, what it does is assign a probability to all possible next words. To build a chatbot, you lay out some text that describes an interaction between a user and a hypothetical 

In [23]:
import faiss
print(faiss.__version__)

1.10.0


In [ ]:
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings

embeddings = FastEmbedEmbeddings(model_name="BAAI/bge-small-en")

# texts = ["Hello, world!", "How are you?"]  For testing ----------------------------------
# embeddings_list = embeddings.embed_documents([docs.page_content for docs in chunks])    |

2025-04-29 20:37:48.219 | ERROR    | fastembed.common.model_management:download_model:430 - Could not download model from HuggingFace: 401 Client Error: Unauthorized for url: https://huggingface.co/api/models/Qdrant/bge-small-en (Request ID: Root=1-6810eb55-08c234e24b40472755ac7656;83fb9b7c-63b0-4727-8ad1-84064c6d5245)

Invalid credentials in Authorization header Falling back to other sources.


In [43]:
huggingaFceApikey =  os.getenv("HUGGINGFACEHUB_API_TOKEN")

vector_store = FAISS.from_documents(chunks,embeddings)
vector_store

In [44]:
vector_store.index_to_docstore_id # for see IDs of stored embeddings, and their corresponding index in the vecotor store(FAISS)

{0: 'f985bc92-9452-4ee3-87c0-1f46e4ac8d62',
 1: 'b4924a6f-2fca-4129-874a-50062864c607',
 2: '19fa8ce6-2d5a-4a31-a00c-bab567d952a7',
 3: 'feb5726c-f893-40f6-a29f-a277e6165110',
 4: '96ed0023-62a7-4a20-abf9-047d3ce9eb39',
 5: 'c0485f68-b549-4dfc-8c6b-766f6ace6193',
 6: '4a4c299a-e5d6-4f35-bccb-082cc7691ae6',
 7: 'a1d4cc6d-92cf-46a7-a916-c9048f27fec7',
 8: '072646c2-b8a7-41e4-97f9-0e1fa76a6a07',
 9: 'fa466aed-09d2-451e-8c2e-c8de876acbe8'}

In [45]:
vector_store.get_by_ids(['e0a79c8b-50a1-4183-b006-39bbed4498aa'])

[]

# Step 2 -> Retrieval

In [46]:
retriever = vector_store.as_retriever(search_type="similarity",search_kwargs={"k": 4})

In [47]:
retriever = vector_store.as_retriever(search_type="similarity",search_kwargs={"k": 4})

In [48]:
retriever

VectorStoreRetriever(tags=['FAISS', 'FastEmbedEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000016D0B326FD0>, search_kwargs={'k': 4})

In [ ]:
question = 'What this video related to?'
context_text = retriever.invoke(question)
context_text

[Document(id='fa466aed-09d2-451e-8c2e-c8de876acbe8', metadata={}, page_content='the other steps in a transformer. Also, on my second channel I just posted a talk I gave a couple months ago about this topic for the company TNG in Munich. Sometimes I actually prefer the content I make as a casual talk rather than a produced video, but I leave it up to you which one of these feels like the better follow-on.'),
 Document(id='072646c2-b8a7-41e4-97f9-0e1fa76a6a07', metadata={}, page_content="next word. Although researchers design the framework for how each of these steps work, it's important to understand that the specific behavior is an emergent phenomenon based on how those hundreds of billions of parameters are tuned during training. This makes it incredibly challenging to determine why the model makes the exact predictions that it does. What you can see is that when you use large language model predictions to autocomplete a prompt, the words that it generates are uncannily fluent, fascin

# Step 3 -> Augmentation

In [50]:
prompt = PromptTemplate(
    template="""You are an assistant for question-answering tasks. Use the following pieces of retrieved context(retrieved from one youtube video) to answer the question. If you don't know the answer, just say that you don't know.
    \nnQuestion: {question}
    \nnRetrieved Context: {context}  """,
    input_variables=["question", "context"],
)



In [51]:
context = '\n\n'.join([doc.page_content for doc in context_text])

In [52]:
final_prompt = prompt.invoke({'context':context,'question':question})
final_prompt

StringPromptValue(text="You are an assistant for question-answering tasks. Use the following pieces of retrieved context(retrieved from one youtube video) to answer the question. If you don't know the answer, just say that you don't know.\n    \nnQuestion: What this video related to?\n    \nnRetrieved Context: the other steps in a transformer. Also, on my second channel I just posted a talk I gave a couple months ago about this topic for the company TNG in Munich. Sometimes I actually prefer the content I make as a casual talk rather than a produced video, but I leave it up to you which one of these feels like the better follow-on.\n\nnext word. Although researchers design the framework for how each of these steps work, it's important to understand that the specific behavior is an emergent phenomenon based on how those hundreds of billions of parameters are tuned during training. This makes it incredibly challenging to determine why the model makes the exact predictions that it does. W

# Generation

In [53]:
# make llm for genration


In [54]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro",api_key=os.getenv('GOOGLE_API_KEY'))

In [55]:
answer = llm.invoke(final_prompt)
answer

AIMessage(content='This video discusses large language models, specifically how they work as chatbots by predicting the next word in a sequence.  It also covers the training process involved, including pre-training and reinforcement learning with human feedback.  The video creator mentions other content on their channels that delve deeper into the technical details of transformers and attention mechanisms within deep learning.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-pro-002', 'safety_ratings': []}, id='run-1971cf51-1fe1-45ec-9079-a975d034665d-0', usage_metadata={'input_tokens': 736, 'output_tokens': 70, 'total_tokens': 806, 'input_token_details': {'cache_read': 0}})

In [56]:
answer.content

'This video discusses large language models, specifically how they work as chatbots by predicting the next word in a sequence.  It also covers the training process involved, including pre-training and reinforcement learning with human feedback.  The video creator mentions other content on their channels that delve deeper into the technical details of transformers and attention mechanisms within deep learning.'

In [57]:
parser = StrOutputParser()

# making chains

In [58]:
def get_context(context_text):
    context = '\n\n'.join([doc.page_content for doc in context_text])
    return context

In [59]:

# Parallel chain
parallelChain = RunnableParallel({
    'context': retriever | RunnableLambda(get_context),
    'question':RunnablePassthrough()

})

chain = parallelChain | prompt | llm | parser
chain.invoke('I would like if you summarize this video')

'This video discusses large language models, specifically focusing on how they predict the next word in a sequence.  It explains that the models use two main operations: attention and feed-forward neural networks. These operations are iterated many times, enriching the data to encode information necessary for accurate predictions.  The training process involves a massive scale of computation, taking over 100 million years at a rate of one billion calculations per second. The video also mentions two types of training: pre-training, where the model learns to autocomplete text passages, and reinforcement learning with human feedback, which refines the model to be a more helpful and less problematic AI assistant.  The creator also plugs content on their other channels that goes into more detail about transformers and attention.'